In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import seaborn as sns; sns.set()

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

In [3]:
materials_path='materials.csv'
matpost_path='matpost.csv'

In [4]:
labels_dict={'code':'CODE',
             'mat_type':'MATERIAL_TYPE',
             'name':'NAME',
             'mat_code':'MAT_CODE',
             'names':'Names',
             'tran_date':'TRAN_DATE',
             'purchase':'Purchase',
             'issue':'Issue',
             'qty':'QTY',
             'compunit_id':'COMPUNIT_ID',
             'qty_nos':'QTY_NOS',
             'mat_qty':'MAT_QTY',
             'mat_rate':'MAT_RATE',
             'mat_value':'MAT_VALUE'}

In [5]:
def matfunc(path1,path2,dlabels):
   
    materials = pd.read_csv(path1)
    matpost = pd.read_csv(path2)
    d=materials.set_index(dlabels.get('code'))[dlabels.get('name')].to_dict()
    matpost[dlabels.get('mat_code')] = matpost[dlabels.get('mat_code')].replace(np.nan, 0)
    matpost[dlabels.get('mat_code')].astype(int)
    def names(s):
        for k,v in d.items():
            if k == s:
                return v    
    matpost[dlabels.get('names')] = matpost[dlabels.get('mat_code')].astype(int).apply(names)
    #matpost.head()
    
    b=materials.set_index(dlabels.get('code'))[dlabels.get('mat_type')].to_dict()
    def mattype(s):
        for k,v in b.items():
            if k == s:
                return v    
    matpost[dlabels.get('mat_type')] = matpost[dlabels.get('mat_code')].astype(int).apply(mattype)
    
    matpost[dlabels.get('tran_date')]= pd.to_datetime(matpost[dlabels.get('tran_date')], format="%Y%m%d")
#matpost.head()
    matpost[dlabels.get('purchase')]=None
    matpost[dlabels.get('issue')]=None
    matpost[dlabels.get('issue')] = matpost[dlabels.get('qty')].apply(lambda x:x if x < 0 else 0)
    matpost[dlabels.get('purchase')] = matpost[dlabels.get('qty')].apply(lambda x:x if x > 0 else 0)
    matpost[dlabels.get('issue')] = matpost['Issue'].abs()
    mat1 = matpost.filter([dlabels.get('tran_date'),dlabels.get('mat_type'),dlabels.get('names'),dlabels.get('compunit_id'),dlabels.get('qty'),dlabels.get('qty_nos'),dlabels.get('mat_qty'),dlabels.get('mat_rate'),dlabels.get('mat_value'),dlabels.get('purchase'),dlabels.get('issue')], axis=1)
    mat1 = mat1.set_index(dlabels.get('tran_date'))
    return mat1

In [6]:
mat1 = matfunc(materials_path,matpost_path,labels_dict)
#mat1

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning:

Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.



In [7]:
mat2 = mat1[mat1['MATERIAL_TYPE'] == 104.0]
#mat2.head()

In [8]:
def mat_by_name(material,plot = True):
    temp = mat1.drop('MATERIAL_TYPE',axis=1)
    temp = mat1[mat1['Names'] == material]
    temp = temp.resample('M').agg({'QTY_NOS':sum,'MAT_QTY':sum,'MAT_RATE':'mean','MAT_VALUE':'mean','Purchase':sum,'Issue':sum})
    if plot == True:
        temp = temp.reset_index('TRAN_DATE')
        return temp.iplot(title=material,kind='scatter',x='TRAN_DATE',y=['MAT_QTY', 'QTY_NOS', 'MAT_RATE', 'MAT_VALUE', 'Purchase','Issue'],mode='lines',size=5)
    else:
        return temp

In [9]:
def get_most(mat,n):
    most_used = mat['Names'].value_counts()[:n].index.tolist()
    return most_used

In [10]:
def name_sort(mat, name):
    def soort(l):
        for i in l.split():
            if i.lower() == name:
                return 1

        return 0
    mat['temp'] = None
    mat['Names'] = mat['Names'].astype(str)
    mat['temp'] = mat['Names'].apply(soort)
    mat = mat[mat['temp'] == 1]
    mat = mat.drop('temp',axis=1)
    return mat

In [11]:
def data_sort(mat_,mat_type='104.0',sort_for_mat_type=False,max_values=5,mat_name='sand',sort_for_name=False):
    
    if sort_for_name == True:
        temp2 = name_sort(mat_,mat_name)
        mat = temp2
        most = get_most(temp2,max_values)
        return interact(mat_by_name, material = most , plot = True)
    
    elif sort_for_mat_type == True:
        temp1 = mat_[mat_['MATERIAL_TYPE'] == float(mat_type)]
        mat = temp1
        most = get_most(temp1,max_values)
        return interact(mat_by_name, material = most , plot = True)
    
    else:
        mat = mat_
        #return mat
        most = get_most(mat,max_values)
        return interact(mat_by_name, material = most , plot = True)

In [12]:
mat2 = mat1[mat1['MATERIAL_TYPE']==104.0]

In [13]:
mat3 = name_sort(mat2,'sand')

In [14]:
most = get_most(mat1,20)

In [15]:
mat_ = mat1
#mat_.head()

In [16]:
data_sort(mat_ = mat1,
          mat_type='104.0',
          sort_for_mat_type=False,
          max_values=15,
          mat_name='sand',
          sort_for_name=True)

interactive(children=(Dropdown(description='material', options=('SAND DRY', 'SAND CHROMITE', 'SAND MANGLORE WA…

<function __main__.mat_by_name(material, plot=True)>